In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

C:\Users\user\AppData\Local\Temp\ipykernel_10672\3573149939.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)


In [6]:
from langchain.llms import OpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
#import os
#import json
#import pandas as pd
#import traceback
#from dotenv import load_dotenv
import PyPDF2


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct" : "correct answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct" : "correct answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct" : "correct answer",
    },
}

In [8]:
TEMPLATE = '''
Text: {text}
You are an expert MCQ maker. Given the above textit isyour job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming to the text as well.
Make sure to format your response like RESPONSE_JSON below an use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

'''

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ['text','number','subject','tone','response_json'],
    template = TEMPLATE
)

In [10]:
mcqChain = LLMChain(llm = llm, prompt=quiz_generation_prompt, output_key= 'quiz', verbose=True)

C:\Users\user\AppData\Local\Temp\ipykernel_10672\3685747232.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  mcqChain = LLMChain(llm = llm, prompt=quiz_generation_prompt, output_key= 'quiz', verbose=True)


In [11]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the Question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at par with the cognitive and analytical abilities of the students,
update the quiz questions which need to be changed and change the tone that it perfectly fits the student's ability.
Quiz_MCQDs:
{quiz}

check from an expert English writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ['subject','quiz'],
    template = TEMPLATE2
)

In [13]:
mcq_review_chain = LLMChain(llm = llm, prompt=quiz_evaluation_prompt, output_key = 'review', verbose= True)

In [14]:
generate_evaluation_chain = SequentialChain(
    chains = [mcqChain, mcq_review_chain], 
    input_variables = ['text','number','subject','tone','response_json'],
    output_variables = ["quiz", "review"], verbose = True)

In [15]:
file_path = r"C:\3MTT\Projects\GenAI\Multi_Choice_Questions_Generator\data.txt"

In [16]:
file_path

'C:\\3MTT\\Projects\\GenAI\\Multi_Choice_Questions_Generator\\data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER = 5
SUBJECT = 'Machine Learning'
TONE= 'simple'

In [23]:
with get_openai_callback() as cb:
    response = generate_evaluation_chain(
        {
            "text": TEXT, # The data source
            "number": NUMBER, # How many MCQs do you want to generate
            "subject": SUBJECT, # 
            "tone": TONE, # Difiulty levels
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\user\AppData\Local\Temp\ipykernel_10672\2481714447.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluation_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied huma

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Machine Learning students.
You need to evaluate the complexity of the Question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at par with the cognitive and analytical abilities of the students,
update the quiz questions which need to be changed and change the tone that it perfectly fits the student's ability.
Quiz_MCQDs:

{
"1": {
"mcq": "Who coined the term machine learning in 1959?",
"options": {
"a": "Donald Hebb",
"b": "Tom M. Mitchell",
"c": "Arthur Samuel",
"d": "Walter Pitts"
},
"correct": "c"
},
"2": {
"mcq": "What was the early machine learning model introduced by Arthur Samuel in the 1950s?",
"options": {
"a": "Program for calculating winning chance in chess",
"b": "Program for analyzing sonar signals",
"c": "Program for recognizing speech patterns",
"d": "Program for classifying cancerous moles"


In [24]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens (input tokens): {cb.prompt_tokens}")
print(f"Completion Tokens (output tokens): {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1905
Prompt Tokens (input tokens): 1427
Completion Tokens (output tokens): 478
Total Cost: 0.0030965


In [25]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [27]:
response.get("quiz")

'\n{\n"1": {\n"mcq": "Who coined the term machine learning in 1959?",\n"options": {\n"a": "Donald Hebb",\n"b": "Tom M. Mitchell",\n"c": "Arthur Samuel",\n"d": "Walter Pitts"\n},\n"correct": "c"\n},\n"2": {\n"mcq": "What was the early machine learning model introduced by Arthur Samuel in the 1950s?",\n"options": {\n"a": "Program for calculating winning chance in chess",\n"b": "Program for analyzing sonar signals",\n"c": "Program for recognizing speech patterns",\n"d": "Program for classifying cancerous moles"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "Who proposed the early mathematical models of neural networks for machine learning algorithms?",\n"options": {\n"a": "Donald Hebb",\n"b": "Raytheon Company",\n"c": "Warren McCulloch",\n"d": "Tom M. Mitchell"\n},\n"correct": "c"\n},\n"4": {\n"mcq": "What was the purpose of Cybertron, the learning machine developed by Raytheon Company in the 1960s?",\n"options": {\n"a": "To analyze weather patterns",\n"b": "To analyze sonar signals, electrocar

In [37]:
quiz = response.get("quiz")
quiz = json.loads(quiz)

In [38]:
quiz_table = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table.append({"MCQ": mcq, "Choices": options, "Corrent": correct})

In [39]:
quiz_table

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a : Donald Hebb | b : Tom M. Mitchell | c : Arthur Samuel | d : Walter Pitts',
  'Corrent': 'c'},
 {'MCQ': 'What was the early machine learning model introduced by Arthur Samuel in the 1950s?',
  'Choices': 'a : Program for calculating winning chance in chess | b : Program for analyzing sonar signals | c : Program for recognizing speech patterns | d : Program for classifying cancerous moles',
  'Corrent': 'a'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks for machine learning algorithms?',
  'Choices': 'a : Donald Hebb | b : Raytheon Company | c : Warren McCulloch | d : Tom M. Mitchell',
  'Corrent': 'c'},
 {'MCQ': 'What was the purpose of Cybertron, the learning machine developed by Raytheon Company in the 1960s?',
  'Choices': 'a : To analyze weather patterns | b : To analyze sonar signals, electrocardiograms, and speech patterns | c : To recognize human faces | d : To play chess',
  'C

In [40]:
quiz_dataframe = pd.DataFrame(quiz_table)

In [41]:
quiz_dataframe

,MCQ,Choices,Corrent
0,Who coined the term machine learning in 1959?,a : Donald Hebb | b : Tom M. Mitchell | c : Ar...,c
1,What was the early machine learning model intr...,a : Program for calculating winning chance in ...,a
2,Who proposed the early mathematical models of ...,a : Donald Hebb | b : Raytheon Company | c : W...,c
3,"What was the purpose of Cybertron, the learnin...",a : To analyze weather patterns | b : To analy...,b
4,"According to Tom M. Mitchell, what is the defi...",a : Improving performance at tasks in T with r...,d


In [42]:
quiz_dataframe.to_csv('Machine_Learning_MCQs.csv')